<a href="https://colab.research.google.com/github/EnPaiva93/tume/blob/colab/tume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon Latin America 2024

## Instalación

In [86]:
%%capture
!pip install groq streamlit ffmpeg-python gTTS

In [25]:
%%capture
!npm install localtunnel

## Utils

In [197]:
def bytesio_to_ogg(input_bytesio: io.BytesIO, input_format: str = "wav", output_file: str = "temp.ogg"):
    """
    Convert audio data from a BytesIO object to an OGG file using ffmpeg-python.

    Args:
        input_bytesio (BytesIO): The input audio data as a BytesIO object.
        output_file (str): The path to save the converted OGG file.
    """
    # Reset the BytesIO pointer to the start
    input_bytesio.seek(0)

    # Use ffmpeg to read from BytesIO and write to an OGG file
    process = (
        ffmpeg
        .input('pipe:0', format=input_format)  # Adjust format if input is not WAV
        .output(
            output_file,
            format='ogg',
            acodec='libopus',  # Correct codec for OGG
            ac=1,              # Mono audio
            b='12k',           # Correct option for bitrate
            application='voip' # Libopus-specific option
        )
        .overwrite_output()
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    # Send BytesIO content to FFmpeg and capture any errors
    stdout, stderr = process.communicate(input=input_bytesio.read())

    if process.returncode != 0:
        raise RuntimeError(f"ffmpeg failed: {stderr.decode('utf-8')}")

    print(f"Converted to OGG: {output_file}")


## Input

In [126]:
# Librería
import uuid
import os, io, subprocess, ffmpeg
from groq import Groq
from google.colab import userdata

from pydantic import BaseModel, ConfigDict, Field

In [20]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [204]:
class Audio(BaseModel):
    audio_bytes: io.BytesIO
    id: str = Field(default=str(uuid.uuid4()))
    audio_path: str = Field(default=None)
    input_format: str = Field(default="wav")
    output_format: str = Field(default="ogg")

    def __init__(self, **data):
        super().__init__(**data)
        self.convert_to_ogg()
        self.audio_path=self.id+"."+self.output_format

    class Config:
        arbitrary_types_allowed = True

    def convert_to_ogg(self):
        """
        Generar el audio temporal en formato ogg
        """
        output_file=self.id+"."+self.output_format
        self.audio_bytes.seek(0)
        bytesio_to_ogg(self.audio_bytes, self.input_format, output_file)

class STT(BaseModel):
  client: Groq = None

  class Config:
        arbitrary_types_allowed = True

  def __init__(self, **data):
        super().__init__(**data)
        self.client = Groq(api_key=GROQ_API_KEY)

  def obtener_texto(self, audio: Audio):
        """
        Transcribes audio and returns the text.
        """

        # Check if the file exists
        if not os.path.exists(audio.audio_path):
            return "Error: The audio file could not be found. Please try again."

        try:
            # Open the OGG file and send it for transcription
            with open(audio.audio_path, "rb") as audio_file:
                transcription = self.client.audio.transcriptions.create(
                    file=audio_file,
                    model="whisper-large-v3-turbo",
                    language="es",
                    temperature=0.0
                )
                return transcription.text
        except Exception:
            # Return a generic error message for Streamlit
            return "An error occurred while processing the audio. Please try again."

In [205]:
stt = STT()

# test

audio_data = io.BytesIO()
with open("temp.wav", "rb") as f:
    audio_data.write(f.read())
audio_data.seek(0)

audio1 = Audio(audio_bytes=audio_data)

result = stt.obtener_texto(audio1)

print(result)

Converted to OGG: c390cebb-4805-407e-8fd0-7882fd3f9720.ogg
 Hola, ¿qué tal? Hola, ¿qué tal?


## Process

In [188]:
tts = gTTS(text, lang='es', tld='com.mx')

In [189]:
fp = io.BytesIO()
tts.write_to_fp(fp)
fp.seek(0)

0

In [194]:
# fp.seek(0)
fp.read()

b''

## Output

In [159]:
from gtts import gTTS
import IPython

In [206]:
class TTS(BaseModel):
  client: gTTS = Field(default=gTTS)

  class Config:
        arbitrary_types_allowed = True

  def generar_audio(self, text) -> Audio:
        """
        Reads text
        """
        tts = self.client(text, lang='es', tld='com.mx')

        fp = io.BytesIO()
        tts.write_to_fp(fp)
        # fp.seek(0)

        return Audio(audio_bytes=fp, input_format="mp3")

In [207]:
# Test
tts = TTS()

text = """La noción más moderna en Occidente de enciclopedia de propósito general y de amplia distribución data de la época de Ephraim Chambers y su Cyclopaedia (1728). Luego le sucederían Denis Diderot y los enciclopedistas del siglo XVIII que se inspiraron en la Cyclopaedia de Chambers, creando el mayor hito del periodo de la Ilustración del siglo XVIII: la Encyclopédie (1751-1772), compilada por un grupo de escritores y hombres de ciencia franceses. De manera plenamente consciente, estos hombres estaban dando la espalda tanto a la religión como a la metafísica como fuentes de conocimiento, viendo en la ciencia pura y su aplicación, la tecnología, una nueva fuerza motriz intelectual según la ideología materialista y laicista propia de la clase media o burguesía entonces en auge. Reunieron en una vasta obra todos los conocimientos científicos de la época, no como un mero registro alfabético, sino como un relato del nuevo modo científico de interpretar el mundo. """

sound = tts.generar_audio(text)

IPython.display.Audio(sound.audio_path, autoplay=True)

Converted to OGG: c390cebb-4805-407e-8fd0-7882fd3f9720.ogg


## UI

In [51]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [60]:
os.getenv('GROQ_API_KEY')

'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'

In [112]:
%%writefile app.py
import os
import streamlit as st
from groq import Groq
from google.colab import userdata

import ffmpeg
from io import BytesIO

# Set page config
st.set_page_config(page_title='Groq Translator', page_icon='🎤')

# Set page title
st.title('Groq Translator')

GROQ_API_KEY = os.getenv('GROQ_API_KEY')

audio_value = st.audio_input("Record a voice message")

client = Groq(api_key=GROQ_API_KEY)

def bytesio_to_ogg(input_bytesio: BytesIO, output_file: str = "temp.ogg"):
    """
    Convert audio data from a BytesIO object to an OGG file using ffmpeg-python.

    Args:
        input_bytesio (BytesIO): The input audio data as a BytesIO object.
        output_file (str): The path to save the converted OGG file.
    """
    # Reset the BytesIO pointer to the start
    input_bytesio.seek(0)

    # Use ffmpeg to read from BytesIO and write to an OGG file
    process = (
        ffmpeg
        .input('pipe:0', format='wav')  # Adjust format if input is not WAV
        .output(
            output_file,
            format='ogg',
            acodec='libopus',  # Correct codec for OGG
            ac=1,              # Mono audio
            b='12k',           # Correct option for bitrate
            application='voip' # Libopus-specific option
        )
        .overwrite_output()
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    # Send BytesIO content to FFmpeg and capture any errors
    stdout, stderr = process.communicate(input=input_bytesio.read())

    if process.returncode != 0:
        raise RuntimeError(f"ffmpeg failed: {stderr.decode('utf-8')}")

    print(f"Converted to OGG: {output_file}")


def obtener_texto(input):
    bytesio_to_ogg(input)
    audio_file= open("temp.ogg", "rb")
    return client.audio.transcriptions.create(
      file=audio_file,
      model="whisper-large-v3-turbo",
      language="es",
      temperature=0.0
    ).text

if audio_value:

  transcription = obtener_texto(audio_value)

  st.write(transcription)

Overwriting app.py


In [113]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://ipv4.icanhazip.com/

34.81.133.21
your url is: https://fast-loops-sell.loca.lt
